In [1]:
"""
Summarizing Sentinel-2 fuel metrics in AFD
Author: maxwell.cook@colorado.edu
"""

import ee
import geemap
import time

ee.Authenticate()

ee.Initialize(project='jfsp-aspen')

print("Success")

Success


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [2]:
def monitor_gee_export(task, sleep=30):
    """ Monitors EE export task """
    while task.active():
        print('Waiting for export to finish..\n\tPatience young padawan.')
        time.sleep(sleep)  # Check every 30 seconds
    
    # Get the status of the task
    status = task.status()
    
    # Check if the task failed or succeeded
    if status['state'] == 'COMPLETED':
        print("Export completed successfully !!!!")
    elif status['state'] == 'FAILED':
        print(f"Export failed! Bummer. Reason: {status.get('error_message', 'Unknown error')}")
    else:
        print(f"Export ended with state: {status['state']}")

print("Functions ready!")

Functions ready!


In [3]:
# Calculate statistics in AFD observations

In [4]:
# Load the exported asset from above
s2metrics = ee.Image('projects/jfsp-aspen/assets/afd_aspen_fires_S2metrics_mosaic')

In [5]:
afds = ee.FeatureCollection('projects/jfsp-aspen/assets/AFD/vnp14img_geo_srm_pix_area_aspenfires')
print(afds.first().propertyNames().getInfo())

['Fire_ID', 'acq_date', 'daynight', 'Ig_Date', 'Last_Date', 'afdID', 'system:index']


In [7]:
# Simplify the feature collection properties.
afds = afds.select(['afdID'])

# Perform the reductions.

mean_med_stdDev = ee.Reducer.mean().combine(
    reducer2=ee.Reducer.stdDev(), sharedInputs=True).combine(
    reducer2=ee.Reducer.median(), sharedInputs=True)

# Perform the reduction by AFD observations
s2_stats = s2metrics.reduceRegions(
    collection=afds,
    reducer=mean_med_stdDev, # Mean and Standard Deviation CBI, CBI_bc, rbr
    scale=30
)

print("Submitted !")

Submitted !


In [8]:
# Export the table. 
s2_stats = s2_stats.map(lambda feature: feature.setGeometry(None)) # drop geometry column

export_task = ee.batch.Export.table.toDrive(
    collection=s2_stats,
    description='vn14img_s2-lai-evi-mndwi',
    fileFormat='CSV', 
    fileNamePrefix='vn14img_s2-lai-evi-mndwi',
    folder='S2'
)

# Start the export task
export_task.start()
print("Export to Earth Engine Asset started!")
# Monitor the task until it's finished
monitor_gee_export(export_task, 120) # print every 2 min

Export to Earth Engine Asset started!
Waiting for export to finish..
	Patience young padawan.
Export completed successfully !!!!
